In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

from initial_functions import *
from scene_function import scenes_responses

pd.set_option('display.max_columns', None) # display all columns
import warnings
warnings.filterwarnings('ignore')

C:\ProgramData\anaconda3\envs\dnp\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


#### character set up 

In [2]:
name = get_name()
race = get_race(races)
profession = get_profession(professions)
weapon = get_weapon(profession)

¿Cuál es tu nombre?:  Ragnar
Elige tu raza: human@, elf@, enan@:  elfo
Elige tu profesión: guerrer@, hechicer@, pícar@:  hechicero


#### character build and sheet

In [3]:
character = {"Nombre":name, 
             "Raza":race,
             "Profesión":profession,
             "Arma":weapon
            }

In [4]:
c_sheet = pd.DataFrame([character])

In [5]:
presentation(c_sheet)

 
Te llaman Ragnar, tejedor de los hilos mágicos del bosque. Tu conexión con la magia antigua es profunda y poderosa. Los espíritus de la naturaleza son tus aliados, y su sabiduría fluye a través de ti.
 
El dragón puede dominar el fuego, pero la antigua magia del bosque responde a tu llamado...


#### trying game flow with text embeddings

In [6]:
class GameEngine:
    
    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.character = None
        self.scenes = {}
        self.responses = {}
        self.similarity_threshold = 0.3
    
    def set_character(self, c_sheet):
        self.character = c_sheet
    
    def add_scene(self, scene_id, scene_text, context=None, required_race=None, required_profession=None):      
        self.scenes[scene_id] = {
            'text': scene_text,
            'context': context,
            'required_race': required_race,
            'required_profession': required_profession,
            'embedding': self.model.encode(scene_text)
        }     
        if scene_id not in self.responses:
            self.responses[scene_id] = []
    
    def add_response(self, scene_id, input_patterns, response_text, race_mod=None, profession_mod=None):
        
        if isinstance(input_patterns, str):
            input_patterns = [input_patterns]
        
        response = {
            'pattern': input_patterns,
            'text': response_text,
            'race_modifier': race_mod,
            'profession_modifier': profession_mod,
            'embeddings': [self.model.encode(pattern) for pattern in input_patterns]
        }
        self.responses[scene_id].append(response)
        
    def find_best_response(self, user_input, current_scene_id):
      
        if current_scene_id not in self.responses:
            return "No hay respuestas disponibles para esta escena."    
        
        user_embedding = self.model.encode(user_input)
        #best_score = self.similarity_threshold
        best_score = -1
        best_response = None
        
        for response in self.responses[current_scene_id]:
            similarities = [np.dot(user_embedding, pattern_embeding)
                            for pattern_embeding in response['embeddings']]
            max_similarity = max(similarities)
            
            if response['race_modifier'] == self.character['Raza'].iloc[0]:
                max_similarity += 0.1
            if response['profession_modifier'] == self.character['Profesión'].iloc[0]:
                max_similarity += 0.1
                
            if max_similarity > best_score:
                best_score = max_similarity
                best_response = response
            
        #return best_response['text'] if best_response else "No entiendo qué quieres hacer. Sé más específico."
        if best_score >= 0.7:
            return best_response["text"]
        else:
            return "Tu acción no está contemplada por ahora en el juego. Sé más específico"
            
    
    def load_scenes(self):
        scenes_responses(self)
        
def play_game():
    
    game = GameEngine()
    game.set_character(c_sheet)
    game.load_scenes()
    
    current_scene_id = 1
    
    time.sleep(1.5)
    print("\n" + game.scenes[current_scene_id]['text'])
    
    while True:
        
        user_input = input("> ").lower()
      
        if user_input == "abandonar juego":
            time.sleep(1.5)
            print ("\n" + f'¡Vuelve pronto o nadie podrá detener al dragón!')
            break
          
        response = game.find_best_response(user_input, current_scene_id)
        print("\n" + response)
        
        if current_scene_id == 1 and "dragón" in response.lower():
            current_scene_id = 2
            time.sleep(1.5)
            print("\n" + game.scenes[current_scene_id]["text"])
            continue
                
        if current_scene_id == 2 and "orbe" in response.lower():
            current_scene_id = 3
            time.sleep(1.5)
            print("\n" + game.scenes[current_scene_id]["text"])
            break
            
        if current_scene_id == 2 and "con tu muerte" in response.lower():
            current_scene_id = 4
            time.sleep(1.5)
            print("\n" + game.scenes[current_scene_id]["text"])
            break
            
        time.sleep(1)
        print("\n¿Qué más quieres hacer?")
        

if __name__ == "__main__":
    play_game()


La caza del dragón negro te lleva hacia una montaña, en donde según rumores se encuentra la guarida de la temible criatura. Para escalar a la cumbre, primero tendrás que atravesar un bosque que la rodea. Puede que de camino encuentres algo útil para tu enfrentamiento... ¿Qué haces?


>  buscar comida



Tu conexión con la magia antigua te atrae hacia un lago sagrado. Un artefacto élfico brilla con poder en su lecho

¿Qué más quieres hacer?


>  nado hacia el



La magia antigua del lago te guía hacia un artefacto de inmenso poder. Al sostener el orbe arcano, las energías fluyen entre ambos, revelando su propósito: es un amplificador mágico diseñado para la destrucción de dragones. Tu victoria está ahora más cerca

¿Qué más quieres hacer?


...


Tu conexión con la magia antigua te revela un sendero élfico olvidado que asciende hacia la montaña. Las energías arcanas te indican que este camino te llevará directamente a la guarida del dragón

Tras ascender por el sendero, llegas a la cima de la montaña. Ante ti se alza una inmensa cueva, y el suelo está cubierto de huesos calcinados. Un rugido estremecedor resuena desde el interior, y una enorme figura emerge entre las sombras. El dragón, con escamas como acero negro y ojos de fuego, se alza ante ti. ¿Qué haces?


>  luchar



Invocas tu antigua magia élfica con precisión. El dragón responde con un rugido ensordecedor y una llamarada que neutralizas con un encantamiento ancestral. Comprendes que ni siquiera la magia más antigua será suficiente por sí sola.

¿Qué más quieres hacer?


>  utilizar el orbe



Tejes tu antigua magia élfica a través del orbe, creando un vínculo con poderes primordiales. La combinación de tu sabiduría arcana y el artefacto genera una cascada de energía que sobrepasa incluso la resistencia mágica del dragón.

¡Lo has conseguido! El ancestral dragón negro, terror de las Montañas del Crepúsculo, ha caído ante tu determinación y coraje. Sus últimos rugidos resuenan entre los picos nevados mientras su cuerpo se desvanece en cenizas llevadas por el viento. Tu nombre será recordado en canciones y leyendas mientras existan estrellas en el cielo y viento en las montañas.
